In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/hwd/"
sz=224
arch=resnet34
bs=64

In [4]:

import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        #for f in files:
            #print('{}{}'.format(subindent, f))


In [8]:
PATH = "data/hwd/"
list_files(PATH)

/
test/
.ipynb_checkpoints/
valid/
    6/
    9/
    7/
    1/
    5/
    4/
    0/
    2/
    8/
    3/
train/
    6/
    9/
    7/
    1/
    5/
    4/
    0/
    2/
    8/
    3/
models/
tmp/
    x_act_test_resnet34_0_224.bc/
        meta/
        data/
    x_act_val_resnet34_0_224.bc/
        meta/
        data/
    x_act_resnet34_0_224.bc/
        meta/
        data/


In [19]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz),test_name='test')

In [21]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 438/438 [02:02<00:00,  3.58it/s]


In [11]:
learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.2922  0.17754 0.94555]                            
[1.      0.21623 0.13856 0.95452]                            
[2.      0.19602 0.12537 0.95872]                            



In [24]:
learn.precompute=False

In [25]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [26]:
learn.fit(lr, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.0608  0.04039 0.98764]                             
[1.      0.03306 0.02712 0.99079]                             
[2.      0.02635 0.02716 0.99207]                             



In [27]:
learn.save('hwd')

In [28]:
learn.load('hwd')

In [29]:
log_preds,y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), axis=0)
#accuracy(probs,y), metrics.log_loss(y, probs)

ValueError: y_true contains only one label (0.0). Please provide the true labels explicitly through the labels argument.

In [32]:
probs[:10]

array([[0.     , 0.     , 0.00001, 0.00001, 0.00001, 0.     , 0.     , 0.00001, 0.99997, 0.     ],
       [0.00021, 0.00001, 0.99676, 0.00044, 0.00006, 0.00003, 0.00043, 0.00014, 0.00178, 0.00014],
       [0.00002, 0.     , 0.     , 0.00004, 0.00001, 0.00002, 0.00001, 0.     , 0.99989, 0.     ],
       [0.0004 , 0.00043, 0.00012, 0.00002, 0.00017, 0.00003, 0.     , 0.99849, 0.00004, 0.00031],
       [0.     , 0.99998, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.99991, 0.00001, 0.00001, 0.00001, 0.00001, 0.     , 0.00002, 0.00001, 0.00002, 0.00001],
       [0.     , 0.99999, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.00006, 0.00001, 0.0002 , 0.00005, 0.     , 0.99894, 0.00066, 0.     , 0.00006, 0.00001],
       [0.00131, 0.00026, 0.00204, 0.00022, 0.00086, 0.00028, 0.00029, 0.00006, 0.97885, 0.01582],
       [0.00003, 0.     , 0.     , 0.00002, 0.00001, 0.00002, 0.00001, 0.     , 0.99991, 0.     ]],
      dty

In [37]:
lab=np.argmax(probs,axis=1)
lab

array([8, 2, 8, ..., 3, 9, 2])

In [34]:
files=data.test_dl.dataset.fnames
files=np.core.defchararray.replace(files,'.jpg','')
files=np.core.defchararray.replace(files,'test/','')
files[:5]
files

array(['7596', '1677', '749', ..., '12660', '6922', '9922'], dtype='<U5')

In [38]:
df = pd.DataFrame({"id":files,"label":lab})

In [40]:
SUBM = f'{PATH}sub/'
os.makedirs(SUBM, exist_ok=True)

In [41]:
df.to_csv(f'{SUBM}hwd.gz', compression='gzip',index=False)

In [42]:
FileLink(f'{SUBM}hwd.gz')

/home/vaibhavnandwani/fastai/courses/dl1/data/hwd/sub/hwd.gz

In [45]:
from IPython.display import Image
Image("data/hwd/test/3.jpg")